# Debugging Infeasibility Issue

This notebook will help identify why the model is infeasible.

In [15]:
import pulp
from pprint import pprint

## Step 1: Load Parameters

In [16]:
%run load_parameters.py

Programs (P): ['CS_Y1', 'CS_Y2', 'DS_Y1', 'DS_Y2']
Courses (C): ['BCS1220', 'BCS1440', 'BCS1530', 'BCS2110', 'BCS2210', 'BCS2720', 'KEN1440', 'KEN1530', 'KEN1220', 'KEN2230', 'KEN2240', 'KEN2530']
Rooms (R): ['EDP150', 'C0.004', 'C0.008', 'C0.016', 'C0.020', 'C1.005', 'C1.015', 'C2.007']
Lecturers (L): ['Tom', 'Ottie', 'Filippe', 'Tony', 'Francessco', 'Tang', 'Marieke', 'Stefan', 'Yan', 'Ashish 10', 'Charis']

Student counts (n_p): {'CS_Y1': 220, 'CS_Y2': 165, 'DS_Y1': 180, 'DS_Y2': 218}

Courses per program (Courses_p): {'CS_Y1': ['BCS1220', 'BCS1440', 'BCS1530'], 'CS_Y2': ['BCS2110', 'BCS2210', 'BCS2720'], 'DS_Y1': ['KEN1220', 'KEN1440', 'KEN1530'], 'DS_Y2': ['KEN2230', 'KEN2240', 'KEN2530']}

Course timelines (Timeline_c): {'BCS1220': ['C', 'C', 'T'], 'BCS1440': ['C', 'C', 'T'], 'BCS1530': ['C', 'C', 'T'], 'BCS2110': ['C', 'C', 'T'], 'BCS2210': ['C', 'C', 'T'], 'BCS2720': ['C', 'C', 'T'], 'KEN1440': ['C', 'C', 'T'], 'KEN1530': ['C', 'C', 'T'], 'KEN1220': ['C', 'C', 'T'], 'KEN2230': 

## Step 2: Run Diagnostic Checks

In [17]:
%run debug_infeasibility.py

diagnose_infeasibility(params)

This script should be run from a notebook after loading parameters:
  %run load_parameters.py
  %run debug_infeasibility.py
  diagnose_infeasibility(params)

INFEASIBILITY DIAGNOSIS

1. ROOM CAPACITY CHECK
--------------------------------------------------------------------------------
BCS1220: 220 students (need ≥110 capacity)
  Valid rooms: ['EDP150', 'C0.004', 'C0.016', 'C0.020']
BCS1440: 220 students (need ≥110 capacity)
  Valid rooms: ['EDP150', 'C0.004', 'C0.016', 'C0.020']
BCS1530: 220 students (need ≥110 capacity)
  Valid rooms: ['EDP150', 'C0.004', 'C0.016', 'C0.020']
BCS2110: 165 students (need ≥82 capacity)
  Valid rooms: ['EDP150', 'C0.004', 'C0.016', 'C0.020']
BCS2210: 165 students (need ≥82 capacity)
  Valid rooms: ['EDP150', 'C0.004', 'C0.016', 'C0.020']
BCS2720: 165 students (need ≥82 capacity)
  Valid rooms: ['EDP150', 'C0.004', 'C0.016', 'C0.020']
KEN1440: 180 students (need ≥90 capacity)
  Valid rooms: ['EDP150', 'C0.004', 'C0.016', 'C0.020']
KEN1530: 180 students (n

## Step 3: Check Unavailability Data Format

In [18]:
%run clean_data.py
clean_params = create_clean_params(params)

Run this from notebook:
  %run load_parameters.py
  %run clean_data.py
  clean_params = create_clean_params(params)
  # Then use clean_params instead of params in your model

FILTERING ZERO-STUDENT COURSES

✅ No zero-student courses found. No filtering needed.

Total sessions to schedule: 36


## Step 4: Test Minimal Case

In [19]:
%run test_minimal.py

Creating minimal test case...
Created 16 variables
✓ Constraint 1 added
✓ Constraint 2 added
✓ Constraint 7 added

Solving minimal test...

Result: Optimal

✅ MINIMAL TEST PASSED!

Schedule found:
  Session 0: Day 1, Slot 2, Room ROOM_A
  Session 1: Day 2, Slot 2, Room ROOM_A


## Step 5: Clean Data (Remove Zero-Student Courses)

In [20]:
%run clean_data.py
clean_params = create_clean_params(params)

Run this from notebook:
  %run load_parameters.py
  %run clean_data.py
  clean_params = create_clean_params(params)
  # Then use clean_params instead of params in your model

FILTERING ZERO-STUDENT COURSES

✅ No zero-student courses found. No filtering needed.

Total sessions to schedule: 36


## Step 6: Try Building Model with Clean Data

Now let's try building the model with only courses that have students.

In [21]:
# Use clean_params instead of params
model = pulp.LpProblem("Course_Scheduling_Clean", pulp.LpMinimize)

x = pulp.LpVariable.dicts(
    "schedule",
    [(c, i, r, d, t)
     for c in clean_params['C']  # ← Using clean_params
     for i in clean_params['I'][c]
     for r in clean_params['R']
     for d in clean_params['D']
     for t in clean_params['T']],
    cat='Binary'
)

print(f"Created {len(x)} variables")

# Objective
model += 0, "Feasibility"

Created 5760 variables


### Add Constraints (using clean_params)

In [22]:
# Constraint 1: Each session scheduled once
for c in clean_params['C']:
    for i in clean_params['I'][c]:
        model += (
            pulp.lpSum(
                x[c, i, r, d, t]
                for r in clean_params['R']
                for d in clean_params['D']
                for t in clean_params['T']
            ) == 1,
            f"C1_{c}_i{i}"
        )

print("✓ Constraint 1 added")

✓ Constraint 1 added


In [23]:
# Constraint 2: Room capacity
constraint_count = 0
for c in clean_params['C']:
    min_capacity = 0.5 * clean_params['students_per_course'][c]
    
    for i in clean_params['I'][c]:
        for r in clean_params['R']:
            if clean_params['Cap'][r] >= min_capacity:
                for d in clean_params['D']:
                    for t in clean_params['T']:
                        model += (
                            x[c, i, r, d, t] * clean_params['students_per_course'][c] 
                            <= 2 * clean_params['Cap'][r],
                            f"C2_{c}_i{i}_{r}_d{d}_t{t}"
                        )
                        constraint_count += 1
            else:
                for d in clean_params['D']:
                    for t in clean_params['T']:
                        model += (
                            x[c, i, r, d, t] == 0,
                            f"C2_block_{c}_i{i}_{r}_d{d}_t{t}"
                        )
                        constraint_count += 1

print(f"✓ Constraint 2: {constraint_count} constraints added")

✓ Constraint 2: 5760 constraints added


In [24]:
# Constraint 3: No room double-booking
for r in clean_params['R']:
    for d in clean_params['D']:
        for t in clean_params['T']:
            model += (
                pulp.lpSum(
                    x[c, i, r, d, t]
                    for c in clean_params['C']
                    for i in clean_params['I'][c]
                ) <= 1,
                f"C3_r{r}_d{d}_t{t}"
            )

print("✓ Constraint 3 added")

✓ Constraint 3 added


In [25]:
# Constraint 4: Lecturer unavailability
DAY_NAMES = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday'}

constraint_count = 0
for c in clean_params['C']:
    lecturer = clean_params['Lecturer'][c]
    unavailable_days = clean_params['U'].get(lecturer, [])
    
    for i in clean_params['I'][c]:
        for d in clean_params['D']:
            day_name = DAY_NAMES[d]
            if day_name in unavailable_days:
                for t in clean_params['T']:
                    model += (
                        pulp.lpSum(
                            x[c, i, r, d, t]
                            for r in clean_params['R']
                        ) == 0,
                        f"C4_Unavailable_{c}_i{i}_{lecturer}_d{d}_t{t}"
                    )
                    constraint_count += 1

print(f"✓ Constraint 4: {constraint_count} constraints added")

if constraint_count == 0:
    print("\n⚠️  WARNING: No unavailability constraints were added!")
    print("   Checking unavailability data...")
    print("\n   Raw U data:")
    for l in clean_params['L']:
        unavailable = clean_params['U'].get(l, [])
        if unavailable:
            print(f"     {l}: {unavailable}")

✓ Constraint 4: 0 constraints added

⚠️  WARNING: No unavailability constraints were added!
   Checking unavailability data...

   Raw U data:


In [26]:
# Constraint 5: Lecturer no double-booking
for l in clean_params['L']:
    courses_by_lecturer = [c for c in clean_params['C'] if clean_params['Lecturer'][c] == l]
    
    for d in clean_params['D']:
        for t in clean_params['T']:
            model += (
                pulp.lpSum(
                    x[c, i, r, d, t]
                    for c in courses_by_lecturer
                    for i in clean_params['I'][c]
                    for r in clean_params['R']
                ) <= 1,
                f"C5_{l}_d{d}_t{t}"
            )

print("✓ Constraint 5 added")

✓ Constraint 5 added


In [27]:
# Constraint 6: Student no overlap
for p in clean_params['P']:
    courses = clean_params['Courses'][p]
    
    if len(courses) > 0:  # Only if program has courses
        for d in clean_params['D']:
            for t in clean_params['T']:
                model += (
                    pulp.lpSum(
                        x[c, i, r, d, t]
                        for c in courses
                        for i in clean_params['I'][c]
                        for r in clean_params['R']
                    ) <= 1,
                    f"C6_{p}_d{d}_t{t}"
                )

print("✓ Constraint 6 added")

✓ Constraint 6 added


In [28]:
# Constraint 7: Chronological ordering
for c in clean_params['C']:
    for i in range(len(clean_params['I'][c]) - 1):
        time_i = pulp.lpSum(
            (5 * d + t) * x[c, i, r, d, t]
            for r in clean_params['R']
            for d in clean_params['D']
            for t in clean_params['T']
        )
        
        time_i_plus_1 = pulp.lpSum(
            (5 * d + t) * x[c, i + 1, r, d, t]
            for r in clean_params['R']
            for d in clean_params['D']
            for t in clean_params['T']
        )
        
        model += (
            time_i + 1 <= time_i_plus_1,
            f"C7_{c}_i{i}_before_i{i+1}"
        )

print("✓ Constraint 7 added")

✓ Constraint 7 added


### Solve with Clean Data

In [29]:
print("\nSolving with cleaned data...")
status = model.solve()

print(f"\nStatus: {pulp.LpStatus[status]}")

if status == pulp.LpStatusOptimal:
    print("\n✅ SUCCESS with clean data!")
    
    # Show the schedule
    DAY_NAMES = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday'}
    TIME_NAMES = {1: '08:30-10:30', 2: '11:00-13:00', 3: '13:30-15:30', 4: '16:00-18:00'}
    
    schedule = []
    for (c, i, r, d, t), var in x.items():
        if var.varValue == 1:
            schedule.append({
                'Course': c,
                'Session': i,
                'Room': r,
                'Day': DAY_NAMES[d],
                'Time': TIME_NAMES[t]
            })
    
    print(f"\nScheduled {len(schedule)} sessions:")
    for s in sorted(schedule, key=lambda x: (list(DAY_NAMES.values()).index(x['Day']), list(TIME_NAMES.values()).index(x['Time']))):
        print(f"  {s['Day']:10} {s['Time']:15} | {s['Course']:10} Session {s['Session']} | Room {s['Room']}")
else:
    print("\n❌ Still infeasible with clean data")
    print("   Need to investigate constraints further")


Solving with cleaned data...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/miniconda3/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/8m/_hd_3f8d7gd1yprsfd9_9fy80000gn/T/28894cb89d454a2bb9eca145017ed06d-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/8m/_hd_3f8d7gd1yprsfd9_9fy80000gn/T/28894cb89d454a2bb9eca145017ed06d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6285 COLUMNS
At line 53807 RHS
At line 60088 BOUNDS
At line 65850 ENDATA
Problem MODEL has 6280 rows, 5761 columns and 36000 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.01 seconds
Cgl0002I 2880 variables fixed
Cgl0004I processed model has 420 rows, 2880 columns (2880 integer (2880 of which binary)) and 15120 elements
Cbc0045I No integer variables out of 2880 objects (2880 integer) have costs
Cbc0

In [30]:
# Import visualization
%run visualize_schedule.py

# Display all views (this does everything!)
schedule_df = display_all_views(x, clean_params)

# ## What You'll See:

# ### 1. **Text-Based Views:**
#    - **Summary** - How many sessions, courses, rooms, lecturers
#    - **Schedule by Day** - All Monday classes, then Tuesday, etc.
#    - **Grid View** - See all rooms at each time slot
#    - **Lecturer Schedule** - Each lecturer's teaching times
#    - **Room Schedule** - When each room is used

# ### 2. **Interactive HTML Calendar:**
#    - Beautiful color-coded calendar
#    - Each course has its own color
#    - Hover-friendly cards showing:
#      - Course code & session number
#      - Lecture or Tutorial
#      - Room assignment
#      - Number of students
#      - Lecturer name

# ### 3. **Export Options:**
#    - Save to Excel: `schedule_df.to_excel('my_schedule.xlsx')`
#    - The DataFrame has all the data for further analysis

## Example Output Preview:
# ```
# MONDAY
# ========
# 08:30-10:30:
#   • BCS1530 - Session 0 (Lecture)
#     Room: C0.004 (capacity 120)
#     Lecturer: Filippe
#     Students: 220

# 11:00-13:00:
#   • KEN2240 - Session 0 (Lecture)
#     Room: C0.016 (capacity 128)
#     Lecturer: Ashish
#     Students: 218
# ```


    Visual Schedule Display Tool
    
    Usage in Jupyter notebook:
    
    from visualize_schedule import display_all_views
    
    # After solving your model:
    df = display_all_views(x, clean_params)
    
    # Or use individual functions:
    from visualize_schedule import create_schedule_dataframe, create_html_calendar
    df = create_schedule_dataframe(x, clean_params)
    html = create_html_calendar(df, clean_params)
    display(HTML(html))
    

SCHEDULE SUMMARY
Total sessions scheduled: 36
Courses: 12
Rooms used: 4
Lecturers: 11

WEEKLY SCHEDULE

MONDAY

11:00-13:00:
  • BCS1530 - Session 0 (Lecture)
    Room: C0.004 (capacity 120)
    Lecturer: Filippe
    Students: 220
  • KEN2240 - Session 0 (Lecture)
    Room: C0.016 (capacity 128)
    Lecturer: Ashish
    Students: 218

16:00-18:00:
  • BCS2110 - Session 0 (Lecture)
    Room: C0.016 (capacity 128)
    Lecturer: Tony
    Students: 165
  • KEN1220 - Session 0 (Lecture)
    Room: C0.004 (capacity 120)
    Lecturer: Tom

Time,Monday,Tuesday,Wednesday,Thursday,Friday
08:30-10:30,,BCS1440 - Session 0 Lecture | Room C0.020 220 students Ottie KEN1220 - Session 1 Lecture | Room C0.004 180 students Tom KEN2230 - Session 0 Lecture | Room C0.016 218 students Yan,BCS2210 - Session 1 Lecture | Room EDP150 165 students Francessco KEN1530 - Session 1 Lecture | Room C0.004 180 students Stefan,BCS1220 - Session 0 Lecture | Room C0.020 220 students Tom,BCS1440 - Session 1 Lecture | Room EDP150 220 students Ottie KEN1440 - Session 2 Tutorial | Room C0.004 180 students Marieke KEN2230 - Session 2 Tutorial | Room C0.020 218 students Yan
11:00-13:00,BCS1530 - Session 0 Lecture | Room C0.004 220 students Filippe KEN2240 - Session 0 Lecture | Room C0.016 218 students Ashish,BCS2210 - Session 0 Lecture | Room EDP150 165 students Francessco KEN2240 - Session 1 Lecture | Room C0.020 218 students Ashish,BCS2210 - Session 2 Tutorial | Room C0.020 165 students Francessco KEN2530 - Session 1 Lecture | Room C0.004 218 students Charis,BCS1220 - Session 1 Lecture | Room C0.016 220 students Tom BCS2720 - Session 0 Lecture | Room EDP150 165 students Tang,BCS1440 - Session 2 Tutorial | Room EDP150 220 students Ottie KEN2530 - Session 2 Tutorial | Room C0.004 218 students Charis
13:30-15:30,,BCS1530 - Session 1 Lecture | Room EDP150 220 students Filippe KEN1530 - Session 0 Lecture | Room C0.004 180 students Stefan,BCS2110 - Session 1 Lecture | Room C0.016 165 students Tony KEN1440 - Session 1 Lecture | Room C0.004 180 students Marieke,BCS1220 - Session 2 Tutorial | Room EDP150 220 students Tom BCS2720 - Session 1 Lecture | Room C0.016 165 students Tang KEN2230 - Session 1 Lecture | Room C0.020 218 students Yan,KEN1530 - Session 2 Tutorial | Room C0.016 180 students Stefan
16:00-18:00,BCS2110 - Session 0 Lecture | Room C0.016 165 students Tony KEN1220 - Session 0 Lecture | Room C0.004 180 students Tom,KEN1440 - Session 0 Lecture | Room C0.004 180 students Marieke KEN2530 - Session 0 Lecture | Room C0.020 218 students Charis,KEN1220 - Session 2 Tutorial | Room C0.004 180 students Tom,BCS2720 - Session 2 Tutorial | Room C0.004 165 students Tang,BCS1530 - Session 2 Tutorial | Room EDP150 220 students Filippe BCS2110 - Session 2 Tutorial | Room C0.016 165 students Tony KEN2240 - Session 2 Tutorial | Room C0.004 218 students Ashish
